In [1]:
import numpy as np
import random

In [27]:
#Reference https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/40671.pdf

In [15]:
def compute_alpha(m):
    assert m > 0
    assert m&(m-1) == 0
    if m == 16:
        return 0.673
    elif m == 32:
        return 0.697
    elif m == 64: 
        return 0.709
    else:
        return 0.7213/(1 + 1.079/m)

In [2]:
def count_leading_zeros(s):
    result = 0
    for i in range(len(s)):
        if s[i] != '0':
            return result
        else:
            result = result + 1
    return result

In [28]:
def simple_hll(logs, p):
    assert p <= 16
    
    m = 1 << p
    alpha = compute_alpha(m)
    buckets = np.zeros(m)
    for log in logs:
        hash_code = hash(str(log)) & ((1 << 32) - 1)
        hash_code_str = "{:032b}".format(hash_code)
        leading_bits = hash_code_str[:p]
        trailing_bits = hash_code_str[p:]
        target_bucket_index = int(leading_bits, 2)
        buckets[target_bucket_index] = max(buckets[target_bucket_index], count_leading_zeros(trailing_bits)+1)
    
    estimated = alpha*m*m / np.sum(np.vectorize(lambda x: 0.5**x)(buckets))
    if estimated < 2.5*m:
        return m * np.log(m / np.count_nonzero(buckets == 0))
    else:
        return estimated

In [4]:
## Initialize test data

In [29]:
max_element = 10000000

In [30]:
def generateTestLogs(cardinality, num_logs):
    samples = random.sample(range(1, max_element), cardinality)
    if num_logs > cardinality:
        other_samples_indices = np.random.randint(cardinality, size=num_logs-cardinality)
        other_samples = np.vectorize(lambda i: samples[i])(other_samples_indices)
        samples.extend(other_samples)
    np.random.shuffle(samples)
    return samples

In [33]:
logs = generateTestLogs(105, 1000)

In [34]:
simple_hll(logs, 16)

105.08420402632788